# Data loading 
Next cells are used to load the data we need.

In [1]:
import os

os.environ["MKL_NUM_THREADS"] = "1" # export MKL_NUM_THREADS=1

import pandas as pd
import numpy as np 
import scipy.sparse as sps
import sys
import matplotlib.pyplot as plt
sys.path.append('../../')


In [2]:
#data which contains users (row), items(col) and implicit interaction (data)
dataset = pd.read_csv('../data_train.csv')
dataset

,row,col,data
0,0,10080,1.0
1,0,19467,1.0
2,1,2665,1.0
3,1,7494,1.0
4,1,17068,1.0
...,...,...,...
113263,7945,2476,1.0
113264,7945,12319,1.0
113265,7945,21384,1.0
113266,7946,8699,1.0


# Data pre-processing
Pre-processing of data to check for missing users, items ecc...

In [3]:
users = dataset.row
items = dataset.col
data = dataset.data
URM_all = sps.coo_matrix((data, (users, items)))
URM_all = URM_all.tocsr() #fast row access -> fast access to users 
URM_all.shape

(7947, 25975)

In [4]:
ICM_df = pd.read_csv('../data_ICM_title_abstract.csv')
ICM_df

,row,col,data
0,0,1185,1.015524
1,0,2507,0.459024
2,0,3534,0.227742
3,0,8766,0.501549
4,0,10862,0.297011
...,...,...,...
490686,25974,12554,0.963016
490687,25974,13003,0.104613
490688,25974,16236,0.118760
490689,25974,18797,0.363301


In [5]:
items = ICM_df.row
features = ICM_df.col
data = ICM_df.data
ICM_all = sps.coo_matrix((data, (items, features)))
ICM_all = ICM_all.tocsr() #fast row access -> fast access to users 
ICM_all.shape

(25975, 20000)

In [6]:
from Base.Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8)
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])

In [7]:
from MatrixFactorization.IALSRecommender_implicit import IALSRecommender_implicit

#Trial 22 finished with value: 0.07502862236249128 and parameters: {'n_factors': 867, 'regularization': 0.7451746230098858, 'iterations': 74, 'weight': 0.5512468009549221}. Best is trial 22 with value: 0.07502862236249128.[0m
#printing self map vector:  [0.07496821 0.07736574 0.07275191]
#printing MAP:  0.07502862236249128

#current best parameters. Tune again near these values
#printing MAP:  0.07395163955000743
#{'n_factors': 767, 'regularization': 0.6762816208423306, 'iterations': 69, 'weight': 0.5686851029710857}

#{'n_factors': 779, 'regularization': 0.41829264231590485, 'iterations': 72, 'weight': 0.533979793083854}

URM_train_ICM_all_IALS = sps.vstack([URM_all*(1-0.5293971458364172), ICM_all.T*0.5293971458364172])
recommender_IALS = IALSRecommender_implicit(URM_train_ICM_all_IALS)
recommender_IALS.fit(n_factors = 768, regularization = 0.4489004525533907, iterations=76)

Recommender_Base_Class: URM Detected 2 (0.01 %) cold users.


In [8]:
from GraphBased.RP3betaRecommender import RP3betaRecommender
#already 3 crossed validated these values
URM_train_ICM_all_RP3 = sps.vstack([URM_all, ICM_all.T])
recommender_rp3Beta = RP3betaRecommender(URM_train_ICM_all_RP3)
recommender_rp3Beta.fit(topK=207, alpha = 0.36128728066686633, beta=0.26052832791376884, implicit = False) 
#recommender_rp3Beta.URM_train = URM_train

RP3betaRecommender: URM Detected 2 (0.01 %) cold users.


In [9]:
from SLIM_ElasticNet.SLIMElasticNetRecommender import SLIMElasticNetRecommender

URM_train_ICM_all_SLIM = sps.vstack([URM_all*(1-0.421), ICM_all.T*0.421])
recommender_SLIM = SLIMElasticNetRecommender(URM_train_ICM_all_SLIM)
recommender_SLIM.fit(topK=996, alpha = 0.000182, l1_ratio = 4.459e-06)
recommender_SLIM.W_sparse.data = np.power(recommender_SLIM.W_sparse.data, 0.8546)

SLIMElasticNetRecommender: URM Detected 2 (0.01 %) cold users.


C:\Users\matti\anaconda3\envs\my_env\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:512: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0011173522798344493, tolerance: 0.0005707712844014168
  model = cd_fast.sparse_enet_coordinate_descent(
C:\Users\matti\anaconda3\envs\my_env\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:512: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.004674019291996956, tolerance: 0.0010425166692584753
  model = cd_fast.sparse_enet_coordinate_descent(
C:\Users\matti\anaconda3\envs\my_env\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:512: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.001766833825968206, tolerance: 0.0003115119761787355
  model = cd_fast.sparse_enet_coordinate_descent(
C:\Users\matti\anaconda3\envs\

SLIMElasticNetRecommender: Processed 4343 ( 16.72% ) in 5.00 min. Items per second: 14.47


C:\Users\matti\anaconda3\envs\my_env\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:512: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.000223438604734838, tolerance: 0.0001539211516501382
  model = cd_fast.sparse_enet_coordinate_descent(
C:\Users\matti\anaconda3\envs\my_env\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:512: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.001741107669658959, tolerance: 0.0006375000230036676
  model = cd_fast.sparse_enet_coordinate_descent(
C:\Users\matti\anaconda3\envs\my_env\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:512: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.03563620150089264, tolerance: 0.003089803270995617
  model = cd_fast.sparse_enet_coordinate_descent(
C:\Users\matti\anaconda3\envs\my_

SLIMElasticNetRecommender: Processed 8720 ( 33.57% ) in 10.00 min. Items per second: 14.53


...

SLIMElasticNetRecommender: Processed 21585 ( 83.10% ) in 25.00 min. Items per second: 14.39


C:\Users\matti\anaconda3\envs\my_env\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:512: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0004239977279212326, tolerance: 0.00021323547116480768
  model = cd_fast.sparse_enet_coordinate_descent(
C:\Users\matti\anaconda3\envs\my_env\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:512: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.001971831312403083, tolerance: 0.0006693197647109628
  model = cd_fast.sparse_enet_coordinate_descent(
C:\Users\matti\anaconda3\envs\my_env\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:512: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.012817815877497196, tolerance: 0.001950539881363511
  model = cd_fast.sparse_enet_coordinate_descent(
C:\Users\matti\anaconda3\envs\

SLIMElasticNetRecommender: Processed 25676 ( 98.85% ) in 30.00 min. Items per second: 14.26


C:\Users\matti\anaconda3\envs\my_env\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:512: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3.255767345428467, tolerance: 0.007426316849887371
  model = cd_fast.sparse_enet_coordinate_descent(
C:\Users\matti\anaconda3\envs\my_env\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:512: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0017841014778241515, tolerance: 0.0007116304477676749
  model = cd_fast.sparse_enet_coordinate_descent(
C:\Users\matti\anaconda3\envs\my_env\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:512: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0006866106414236128, tolerance: 0.00022016357979737222
  model = cd_fast.sparse_enet_coordinate_descent(
C:\Users\matti\anaconda3\envs\my

SLIMElasticNetRecommender: Processed 25975 ( 100.00% ) in 30.36 min. Items per second: 14.26


In [10]:
from SLIM_BPR.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython

URM_train_ICM_all_SLIM = sps.vstack([URM_all*(1- 0.6229636472829045), ICM_all.T* 0.6229636472829045])
recommender_SLIM_BPR = SLIM_BPR_Cython(URM_train_ICM_all_SLIM, recompile_cython=False, verbose=False)
recommender_SLIM_BPR.fit(epochs=87, sgd_mode= "adam", learning_rate= 6.12252603066224e-06, topK = 805, batch_size = 35, lambda_i=3.0399585540912118e-05, lambda_j=0.00012176512055401498 )

Unable to read memory status: list index out of range
SLIM_BPR_Recommender: Epoch 1 of 87. Elapsed time 0.09 sec
SLIM_BPR_Recommender: Epoch 2 of 87. Elapsed time 0.19 sec
SLIM_BPR_Recommender: Epoch 3 of 87. Elapsed time 0.29 sec
SLIM_BPR_Recommender: Epoch 4 of 87. Elapsed time 0.39 sec
SLIM_BPR_Recommender: Epoch 5 of 87. Elapsed time 0.49 sec
SLIM_BPR_Recommender: Epoch 6 of 87. Elapsed time 0.61 sec
SLIM_BPR_Recommender: Epoch 7 of 87. Elapsed time 0.71 sec
SLIM_BPR_Recommender: Epoch 8 of 87. Elapsed time 0.81 sec
SLIM_BPR_Recommender: Epoch 9 of 87. Elapsed time 0.90 sec
SLIM_BPR_Recommender: Epoch 10 of 87. Elapsed time 1.00 sec
SLIM_BPR_Recommender: Epoch 11 of 87. Elapsed time 1.12 sec
SLIM_BPR_Recommender: Epoch 12 of 87. Elapsed time 1.21 sec
SLIM_BPR_Recommender: Epoch 13 of 87. Elapsed time 1.31 sec
SLIM_BPR_Recommender: Epoch 14 of 87. Elapsed time 1.40 sec
SLIM_BPR_Recommender: Epoch 15 of 87. Elapsed time 1.48 sec
SLIM_BPR_Recommender: Epoch 16 of 87. Elapsed time 1.57

In [26]:
grouped_users = dataset.groupby(['row']).count()
sorted_users = grouped_users.sort_values(by=['col'], ascending=True)
sorted_users = sorted_users.index.to_numpy()
print(sorted_users)
block_size = int(len(sorted_users)*0.50)

start_pos_group_0 = 0*block_size
end_pos_group_0 = min((0+1)*block_size, len(sorted_users))
users_in_group_0 = sorted_users[start_pos_group_0:end_pos_group_0]
users_not_in_group_0_flag = np.isin(sorted_users, users_in_group_0, invert = True)
users_not_in_group_0 = sorted_users[users_not_in_group_0_flag]
print(users_not_in_group_0)

start_pos_group_1 = 1*block_size
end_pos_group_1 = min((1+1)*block_size, len(sorted_users))
users_in_group_1 = sorted_users[start_pos_group_1:end_pos_group_1]
users_not_in_group_1_flag = np.isin(sorted_users, users_in_group_1, invert = True)
users_not_in_group_1 = sorted_users[users_not_in_group_1_flag]
print(users_not_in_group_1)

[6012 3608 3171 ... 1863 2282 7382]
[6233  112 5662 ... 1863 2282 7382]
[6012 3608 3171 ... 7381 6953 7382]


In [27]:
test_users = pd.read_csv('../data_target_users_test.csv')
test_users

,user_id
0,0
1,1
2,2
3,3
4,4
...,...
7939,7942
7940,7943
7941,7944
7942,7945


In [37]:
user_id = test_users['user_id'].to_numpy()
users_in_group_0 = users_in_group_0[np.isin(users_in_group_0, user_id)]
print(len(users_in_group_0))


users_middle_flag = np.isin(user_id, users_in_group_0, invert = True)
users_in_group_1 = user_id[users_middle_flag]
print(len(users_in_group_1))



3970
3974


In [38]:
#group 0 recommender 
#{'alpha': 0.1128413617931043, 'beta': 0.8018394254802035, 'gamma': 0.054564837202041326, 'phi': 0.873425398015146}

from KNN.ItemKNNScoresHybridNRecommender import ItemKNNScoresHybridNRecommender
URM_train_ICM_all = sps.vstack([URM_all, ICM_all.T])
hybrid_0 = ItemKNNScoresHybridNRecommender(URM_train_ICM_all, [recommender_IALS,recommender_rp3Beta,recommender_SLIM,recommender_SLIM_BPR], 4)
hybrid_0.fit([0.1128413617931043,   0.8018394254802035, 0.054564837202041326, 0.873425398015146])
#evaluator_validation.evaluateRecommender(hybrid)

ItemKNNScoresHybridNRecommender: URM Detected 2 (0.01 %) cold users.
number of recommenders: 4


In [39]:
#group 1 recommender
URM_train_ICM_all = sps.vstack([URM_all, ICM_all.T])
#{'alpha': 0.12662327101812004, 'beta': 0.8154335160223893, 'gamma': 0.08627086061106863, 'phi': 0.6714970158040552}
hybrid_1 = ItemKNNScoresHybridNRecommender(URM_train_ICM_all, [recommender_IALS,recommender_rp3Beta,recommender_SLIM,recommender_SLIM_BPR], 4)
hybrid_1.fit([0.12662327101812004,  0.8154335160223893, 0.08627086061106863,  0.6714970158040552])

ItemKNNScoresHybridNRecommender: URM Detected 2 (0.01 %) cold users.
number of recommenders: 4


In [40]:
recommendations_first = hybrid_0.recommend(users_in_group_0,cutoff = 10)
recommendations_last = hybrid_1.recommend(users_in_group_1,cutoff = 10)

In [41]:
for index in range(len(recommendations_first)):
    recommendations_first[index]=np.array(recommendations_first[index])

print(len(recommendations_first))

for index in range(len(recommendations_last)):
    recommendations_last[index]=np.array(recommendations_last[index])

print(len(recommendations_last))

3970
3974


In [42]:
elementi = test_users['user_id'].to_numpy()
recommendations = []

for index in elementi:
    if index in users_in_group_0:
        local_index, = np.where(users_in_group_0 == index) 
        recommendations.append(recommendations_first[local_index[0]])
    
    if index in users_in_group_1:
        local_index, = np.where(users_in_group_1 == index) 
        recommendations.append(recommendations_last[local_index[0]])
    
    print(index, " - ", local_index[0])

print(recommendations)

0  -  275
1  -  0
2  -  963
3  -  1024
4  -  1
5  -  2
6  -  2690
7  -  3
8  -  1918
9  -  4
10  -  5
11  -  6
12  -  1984
13  -  985
14  -  2203
15  -  7
16  -  1052
17  -  8
18  -  9
19  -  1070
20  -  10
21  -  11
22  -  12
23  -  13
24  -  1415
25  -  14
26  -  3549
27  -  15
28  -  16
29  -  17
30  -  18
31  -  1181
32  -  1080
33  -  2407
34  -  1768
35  -  1765
36  -  19
37  -  20
38  -  21
39  -  1013
40  -  22
41  -  23
42  -  24
43  -  25
44  -  1640
45  -  2753
46  -  26
47  -  27
48  -  28
49  -  2774
50  -  29
51  -  30
52  -  3912
53  -  31
54  -  1044
55  -  1023
56  -  2483
57  -  32
58  -  971
59  -  1836
60  -  3949
61  -  33
62  -  34
63  -  35
64  -  36
65  -  3436
66  -  37
67  -  965
68  -  38
69  -  1007
70  -  1035
71  -  1395
72  -  39
73  -  40
74  -  41
75  -  42
76  -  43
77  -  44
78  -  2734
79  -  2009
80  -  1913
81  -  45
82  -  46
83  -  3921
84  -  47
85  -  1081
86  -  1083
87  -  48
88  -  49
89  -  2079
90  -  50
91  -  3838
92  -  51
93  -  52
94 

In [43]:
'''user_id = test_users['user_id']
recommendations = hybrid.recommend(user_id,cutoff = 10)'''


"user_id = test_users['user_id']\nrecommendations = hybrid.recommend(user_id,cutoff = 10)"

In [44]:
test_users['item_list']= recommendations

test_users['item_list'] = pd.DataFrame([str(line).strip('[').strip(']').replace("'","") for line in test_users['item_list']])

test_users
test_users.to_csv('submission.csv', index=False)
